In [1]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import seaborn as sns

from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier

import matplotlib.pyplot as plt

import numpy as np

from sklearn.preprocessing import StandardScaler

np.random.seed(7)

/home/vampire/PycharmProjects/mikasa_ed/venv/lib/python3.5/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [4]:
def validate_model(cls_name, model, X_train, y_train, X_test, y_test, period, pair, lag):
    info_format = '{3:.3f}: [period: {0}](lag: {1}): {2} | {4}'
    
    model.fit(X_train, y_train)
    score = model.score(X_test, y_test)
    print(info_format.format(period, lag, cls_name, score, pair))
    return score

def test_cls(period, pair, lag):
    df = pd.read_csv('../datasets/{}/{}.csv'.format(period, pair))
    close_lag_fields = ['close-{}'.format(n+1) for n in range(lag)]
    volume_lag_fields = ['volume-{}'.format(n+1) for n in range(lag)]
    df['datetime'] = pd.to_datetime(df['datetime'], unit='s')
    df['hour'] = df['datetime'].dt.hour
    df['dow'] = df['datetime'].dt.weekday
    df = pd.get_dummies(df, columns=['dow', 'hour'])
    dow_fields = [col for col in df if col.startswith('dow')]
    hour_fields = [col for col in df if col.startswith('hour')]
    for n in range(1, lag+1):
        df['close-' + str(n)] = df['close'] - df.shift(n)['close']
        df['volume-' + str(n)] = df['volume'] - df.shift(n)['volume']
    df['up'] = df['close'] < df.shift(-1)['close']
    df.dropna(inplace=True)
    X = df[close_lag_fields + volume_lag_fields + dow_fields + hour_fields].values
    y = df['up'].astype(int).values

    scaler = StandardScaler()

    X = scaler.fit_transform(X)

    X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=0)
    
    best_score = 0.0

    model = XGBClassifier()
    score = validate_model('XGBoost', model, X_train, y_train, X_test, y_test, period, pair, lag)
    if score > best_score:
        best_score = score
    
    print()
    return best_score


In [5]:
best_score = 0.0

for period in ['5min', '30min', 'day']:
    for lag in range(2, 31):
        for pair in ['BTC_ETC', 'BTC_LTC']:
            score = test_cls(period, pair, lag)
            if score > best_score:
                best_score = score
            print('Best score: {:.3f}'.format(best_score))
            print()

0.571: [period: 5min](lag: 2): XGBoost | BTC_ETC

Best score: 0.571

0.574: [period: 5min](lag: 2): XGBoost | BTC_LTC

Best score: 0.574

0.573: [period: 5min](lag: 3): XGBoost | BTC_ETC

Best score: 0.574

0.578: [period: 5min](lag: 3): XGBoost | BTC_LTC

Best score: 0.578

0.573: [period: 5min](lag: 4): XGBoost | BTC_ETC

Best score: 0.578

0.578: [period: 5min](lag: 4): XGBoost | BTC_LTC

Best score: 0.578

0.571: [period: 5min](lag: 5): XGBoost | BTC_ETC

Best score: 0.578

0.585: [period: 5min](lag: 5): XGBoost | BTC_LTC

Best score: 0.585

0.572: [period: 5min](lag: 6): XGBoost | BTC_ETC

Best score: 0.585

0.582: [period: 5min](lag: 6): XGBoost | BTC_LTC

Best score: 0.585

0.569: [period: 5min](lag: 7): XGBoost | BTC_ETC

Best score: 0.585

0.582: [period: 5min](lag: 7): XGBoost | BTC_LTC

Best score: 0.585

0.574: [period: 5min](lag: 8): XGBoost | BTC_ETC

Best score: 0.585

0.580: [period: 5min](lag: 8): XGBoost | BTC_LTC

Best score: 0.585

0.571: [period: 5min](lag: 9): XGB

0.534: [period: day](lag: 4): XGBoost | BTC_LTC

Best score: 0.589

0.542: [period: day](lag: 5): XGBoost | BTC_ETC

Best score: 0.589

0.597: [period: day](lag: 5): XGBoost | BTC_LTC

Best score: 0.597

0.583: [period: day](lag: 6): XGBoost | BTC_ETC

Best score: 0.597

0.528: [period: day](lag: 6): XGBoost | BTC_LTC

Best score: 0.597

0.597: [period: day](lag: 7): XGBoost | BTC_ETC

Best score: 0.597

0.556: [period: day](lag: 7): XGBoost | BTC_LTC

Best score: 0.597

0.472: [period: day](lag: 8): XGBoost | BTC_ETC

Best score: 0.597

0.556: [period: day](lag: 8): XGBoost | BTC_LTC

Best score: 0.597

0.514: [period: day](lag: 9): XGBoost | BTC_ETC

Best score: 0.597

0.444: [period: day](lag: 9): XGBoost | BTC_LTC

Best score: 0.597

0.592: [period: day](lag: 10): XGBoost | BTC_ETC

Best score: 0.597

0.563: [period: day](lag: 10): XGBoost | BTC_LTC

Best score: 0.597

0.493: [period: day](lag: 11): XGBoost | BTC_ETC

Best score: 0.597

0.592: [period: day](lag: 11): XGBoost | BTC_